In [1]:
# 1- Scrap details from Mobile Marketplaces like Phone , url , brand and prices as per the memory and compute. At Least 4 websites this would be handle by via frontend from socket via fastAPI .
# 2- Find sub domains and their valid IPs of a specific domain.
# 3- The Project would follow industry standards like writing tests (Pytest), use a linter (ruff) , the code should be modular and implement micro services.

In [5]:
pip install requests


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.4.26-py3-none-any.whl (159 kB)
   ---------------------------------------- 0.0/105.4 kB ? eta -:--:--
   ----------------------- ---------------- 61.4/105.4 kB 3.4 MB/s eta 0:00:01
   -------------------------- ------------ 71.7/105.4 kB 787.7 kB/s eta 0:00:01
   ------------------------------------ - 102.4/105.4 kB 845.5 kB/s eta 0:00:01
   -------------------------------------- 105.4/105.4 kB 508.9 kB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from  bs4 import BeautifulSoup #pip install beautifulsoup4
import requests

In [18]:
# url = "https://www.whatmobile.com.pk/0_to_150001_Mobiles"
url = "https://www.whatmobile.com.pk/0_to_150001_Mobiles"
# url = "https://www.telemart.pk/mobile-and-tablets/mobile-phone.html"

In [19]:
# creating a session
r_session = requests.session()
r = r_session.get(url=url)
print(r)

<Response [200]>


In [30]:
# Defining a soup
soup = BeautifulSoup(r.text, 'html.parser')

# Find all <td> tags with the specified attributes
product_tags = soup.find_all('td', align="left", colspan="2", nowrap=True, valign="top")

# Extract and display the text content of each product
products = []
base_url = "https://www.whatmobile.com.pk"

for product in product_tags:
    brand = product.find('font', {'color': '#1C477E', 'face': 'Tahoma', 'size': '1'}).b.text.strip() if product.find('font', {'color': '#1C477E', 'face': 'Tahoma', 'size': '1'}) and product.find('font', {'color': '#1C477E', 'face': 'Tahoma', 'size': '1'}).b else None
    model = product.find('p').font.text.strip() if product.find('p') and product.find('p').font else None
    price = product.find('font', {'color': '#000000', 'face': 'verdana', 'size': '1'}).text.strip() if product.find('font', {'color': '#000000', 'face': 'verdana', 'size': '1'}) else None
    product_url = base_url + product.find('a').get('href') if product.find('a') and product.find('a').get('href') else None
    
    if brand and model and price and product_url:
        products.append({'brand': brand, 'model': model, 'price': price, 'url': product_url})

# Display the extracted products
print(products)

[{'brand': 'Alcatel', 'model': 'Pixi 4 6', 'price': '10,000', 'url': 'https://www.whatmobile.com.pk/Alcatel_Pixi-4-6'}, {'brand': 'Alcatel', 'model': 'Idol 5', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_Idol-5'}, {'brand': 'Alcatel', 'model': 'Idol 4 Pro', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_Idol-4-Pro'}, {'brand': 'Alcatel', 'model': 'Shine Lite', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_Shine-Lite'}, {'brand': 'Alcatel', 'model': 'Flash 2', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_Flash-2'}, {'brand': 'Alcatel', 'model': '1x', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_1x'}, {'brand': 'Alcatel', 'model': 'Pixi Power Plus', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_Pixi-Power-Plus'}, {'brand': 'Alcatel', 'model': 'U5 HD', 'price': 'Coming Soon', 'url': 'https://www.whatmobile.com.pk/Alcatel_U5-HD'}, {'brand': 'Alc

In [24]:
# css for product_name
product_name_css = 'tr td.BiggerText'
product_names = soup.select(product_name_css)


# Extracting brand, model, and price
# Extracting product details
products = []

for product in product_names:
    brand = product.select_one('font[b]').text.strip() if product.select_one('font[b]') else None
    model = product.select_one('p font').text.strip() if product.select_one('p font') else None
    price = product.select_one('font[color="#000000"]').text.strip() if product.select_one('font[color="#000000"]') else None
    
    if brand and model and price:
        products.append({'brand': brand, 'model': model, 'price': price})

# Display the extracted products
print(products)

[]


In [ ]:
# getting attribute from html using 'get('<attrib_name>')'

for i in range(len(product_names)):
    product_names[i].select('amp-img')[0].get('src')
    print(product_names[i].select('amp-img')[0].get('src'))

IndexError: list index out of range

In [41]:
# Find sub domains and their valid IPs of a specific domain
import socket
import requests
from bs4 import BeautifulSoup
import tldextract
import dns.resolver # pip install dnspython
import re
import json

# Function to find subdomains of a given domain
def find_subdomains(domain):
    subdomains = []
    try:
        # Use a DNS resolver to find subdomains
        answers = dns.resolver.resolve(domain, 'NS')
        for answer in answers:
            subdomain = str(answer.target).rstrip('.')
            subdomains.append(subdomain)
    except Exception as e:
        print(f"Error resolving {domain}: {e}")
    return subdomains

# Function to get valid IPs for a list of subdomains
def get_valid_ips(subdomains):
    valid_ips = {}
    for subdomain in subdomains:
        try:
            ip = socket.gethostbyname(subdomain)
            valid_ips[subdomain] = ip
        except socket.gaierror:
            print(f"Could not resolve {subdomain}")
    return valid_ips

# Function to scrape subdomains from a website

def scrape_subdomains(url):
    subdomains = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract subdomains using regex or specific HTML tags
        for link in soup.find_all('a', href=True):
            href = link['href']
            if re.match(r'^https?://', href):
                subdomain = tldextract.extract(href).subdomain
                if subdomain and subdomain not in subdomains:
                    subdomains.append(subdomain)
    except Exception as e:
        print(f"Error scraping {url}: {e}")
    return subdomains


# Main function to find subdomains and their valid IPs
def main(domain):
    # Find subdomains
    subdomains = find_subdomains(domain)
    print(f"Subdomains of {domain}: {subdomains}")

    # Get valid IPs for the subdomains
    valid_ips = get_valid_ips(subdomains)
    print(f"Valid IPs for subdomains: {valid_ips}")

    # Scrape subdomains

    url = f"https://{domain}"
    scraped_subdomains = scrape_subdomains(url)
    print(f"Scraped subdomains from {url}: {scraped_subdomains}")

    # Combine scraped subdomains with found subdomains
    all_subdomains = list(set(subdomains + scraped_subdomains))
    print(f"All subdomains: {all_subdomains}")

    # Get valid IPs for all subdomains
    all_valid_ips = get_valid_ips(all_subdomains)
    print(f"Valid IPs for all subdomains: {all_valid_ips}")

if __name__ == "__main__":
    domain = "ubank.com.pk"  # Replace with the target domain
    main(domain)

Subdomains of ubank.com.pk: ['ram.ns.cloudflare.com', 'vida.ns.cloudflare.com']
Valid IPs for subdomains: {'ram.ns.cloudflare.com': '173.245.59.225', 'vida.ns.cloudflare.com': '108.162.192.236'}
Scraped subdomains from https://ubank.com.pk: ['online', 'www', 'web', 'play', 'apps']
All subdomains: ['ram.ns.cloudflare.com', 'www', 'play', 'online', 'apps', 'web', 'vida.ns.cloudflare.com']
Could not resolve www
Could not resolve play
Could not resolve online
Could not resolve apps
Could not resolve web
Valid IPs for all subdomains: {'ram.ns.cloudflare.com': '173.245.59.225', 'vida.ns.cloudflare.com': '108.162.192.236'}


In [ ]:
import subprocess
import requests

# Function to run nmap and get subdomains and IPs
def get_subdomains_and_ips(domain):
    subdomains_ips = []
    try:
        # Run nmap command to find subdomains and IPs
        result = subprocess.run(['nmap', '-p80', '--script', 'http-title', domain], stdout=subprocess.PIPE, text=True)
        output = result.stdout

        # Parse the nmap output to extract subdomains and IPs
        for line in output.splitlines():
            if "Nmap scan report for" in line:
                parts = line.split()
                if len(parts) > 4:
                    subdomains_ips.append(parts[4])  # Extract IP or domain
    except Exception as e:
        print(f"Error running nmap: {e}")
    return subdomains_ips

# Function to check HTTP status of IPs
def check_http_status(ips):
    valid_ips = []
    for ip in ips:
        try:
            response = requests.get(f"http://{ip}", timeout=5)
            if response.status_code == 200:
                valid_ips.append(ip)
        except Exception as e:
            print(f"Error connecting to {ip}: {e}")
    return valid_ips

# Main function
def main(domain):
    # Get subdomains and IPs using nmap
    subdomains_ips = get_subdomains_and_ips(domain)
    print(f"Subdomains and IPs: {subdomains_ips}")

    # Check HTTP status of IPs
    valid_ips = check_http_status(subdomains_ips)
    print(f"Valid IPs with 200 status: {valid_ips}")

    # Write valid IPs to a text file
    with open("valid_ips.txt", "w") as file:
        for ip in valid_ips:
            file.write(ip + "\n")
    print("Valid IPs saved to valid_ips.txt")

# Replace 'example.com' with your target domain
main("ubank.com.pk")





Error running nmap: [WinError 2] The system cannot find the file specified
Subdomains and IPs: []
Valid IPs with 200 status: []
Valid IPs saved to valid_ips.txt
